In [ ]:
%%capture
import os
import site
os.sys.path.insert(0, '/home/schirrmr/code/reversible/reversible2/')
os.sys.path.insert(0, '/home/schirrmr/braindecode/code/braindecode/')
os.sys.path.insert(0, '/home/schirrmr/code/explaining/reversible//')
%cd /home/schirrmr/


%load_ext autoreload
%autoreload 2
import numpy as np
import logging
log = logging.getLogger()
log.setLevel('INFO')
import sys
logging.basicConfig(format='%(asctime)s %(levelname)s : %(message)s',
                     level=logging.INFO, stream=sys.stdout)
import matplotlib
from matplotlib import pyplot as plt
from matplotlib import cm
%matplotlib inline
%config InlineBackend.figure_format = 'png'
matplotlib.rcParams['figure.figsize'] = (12.0, 1.0)
matplotlib.rcParams['font.size'] = 14
from matplotlib import rcParams, cycler
import seaborn
seaborn.set_style('darkgrid')

from reversible.sliced import sliced_from_samples

from numpy.random import RandomState

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import numpy as np
import copy
import math

import itertools
from reversible.plot import create_bw_image
import torch as th
from braindecode.torch_ext.util import np_to_var, var_to_np
from reversible.revnet import ResidualBlock, invert, SubsampleSplitter, ViewAs, ReversibleBlockOld
from spectral_norm import spectral_norm
from conv_spectral_norm import conv_spectral_norm

def display_text(text, fontsize=18):
    fig = plt.figure(figsize=(12,0.1))
    plt.title(text, fontsize=fontsize)
    plt.axis('off')
    display(fig)
    plt.close(fig)

In [ ]:
from braindecode.datasets.bbci import BBCIDataset
from braindecode.mne_ext.signalproc import mne_apply
# we loaded all sensors to always get same cleaning results independent of sensor selection
# There is an inbuilt heuristic that tries to use only EEG channels and that definitely
# works for datasets in our paper
#train_loader = BBCIDataset('/data/schirrmr/schirrmr/HGD-public/reduced/train/13.mat')
#test_loader = BBCIDataset('/data/schirrmr/schirrmr/HGD-public/reduced/test/13.mat')
start_cnt = BBCIDataset('/data/schirrmr/schirrmr/HGD-public/reduced/train/4.mat',).load()
start_cnt = start_cnt.drop_channels(['STI 014'])
def car(a):
    return a - np.mean(a, keepdims=True, axis=0)

start_cnt = mne_apply(
    car, start_cnt)

start_cnt = start_cnt.reorder_channels(['C3', 'C4'])

In [ ]:
from collections import OrderedDict
from braindecode.datautil.trial_segment import create_signal_target_from_raw_mne

marker_def = OrderedDict([('Right Hand', [1]), ('Left Hand', [2],),
                         ('Rest', [3]), ('Feet', [4])])
ival = [500,1500]
from braindecode.mne_ext.signalproc import mne_apply, resample_cnt
from braindecode.datautil.signalproc import exponential_running_standardize, bandpass_cnt

log.info("Resampling train...")
cnt = resample_cnt(start_cnt, 250.0)
log.info("Standardizing train...")
cnt = mne_apply(lambda a: exponential_running_standardize(a.T ,factor_new=1e-3, init_block_size=1000, eps=1e-4).T,
                     cnt)
cnt = resample_cnt(cnt, 32.0)
cnt = resample_cnt(cnt, 64.0)
#cnt = mne_apply(
#    lambda a: bandpass_cnt(a, 0, 2, cnt.info['sfreq'],
#                           filt_order=10,
#                           axis=1), cnt)

train_set = create_signal_target_from_raw_mne(cnt, marker_def, ival)
cnt_bandpassed =  mne_apply(
    lambda a: bandpass_cnt(a, 8, 13, cnt.info['sfreq'],
                           filt_order=10,
                           axis=1), cnt)
alpha_set = create_signal_target_from_raw_mne(cnt_bandpassed, marker_def, ival)
x_alpha_right = alpha_set.X[alpha_set.y == 0]

x_alpha_rest = alpha_set.X[alpha_set.y == 2]

alpha_a = np_to_var(x_alpha_right[:160,0:1,:,None], dtype=np.float32)

alpha_b = np_to_var(x_alpha_rest[:160,0:1,:,None], dtype=np.float32)
inputs_alpha = [alpha_a, alpha_b]


In [ ]:
plt.figure(figsize=(8,3))
plt.plot(var_to_np(inputs_alpha[0][:3]).squeeze().T);

In [ ]:
def rev_block(n_c, n_i_c):
     return ReversibleBlockOld(
        nn.Sequential(
            nn.Linear(n_c // 2, n_i_c,),
            nn.ReLU(),
            nn.Linear(n_i_c, n_c // 2,)),
        nn.Sequential(
            nn.Linear(n_c // 2, n_i_c,),
            nn.ReLU(),
            nn.Linear(n_i_c, n_c // 2))
    )

In [ ]:
fig = plt.figure(figsize=(12,12))
emp_cov = np.cov(var_to_np(inputs_alpha[0]).squeeze().T)
plt.imshow(emp_cov, cmap=cm.coolwarm, vmin=-np.max(np.abs(emp_cov)),
          vmax=np.max(np.abs(emp_cov)))
cbar = plt.colorbar()
cbar.set_label("Real Covariance")
display(fig)
plt.close(fig)

In [ ]:
fig = plt.figure(figsize=(12,12))
emp_cov = np.abs(np.cov(var_to_np(inputs_alpha[0]).squeeze().T))
plt.imshow(emp_cov, cmap=cm.coolwarm, vmin=0,
          vmax=np.percentile(np.abs(emp_cov), 30))
cbar = plt.colorbar()
cbar.set_label("Real Covariance")
display(fig)
plt.close(fig)